In [1]:
#Importamos algunas cosas

import numpy as np
import matplotlib.pyplot as plt

from qiskit.providers.ibmq import least_busy
import qiskit
from qiskit.visualization import plot_histogram

# Needed for functions
import time

import qiskit.quantum_info as qi
from qiskit.providers.aer import noise
from qiskit.compiler import assemble

from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter
from qiskit.circuit.library import XGate, YGate, ZGate, U1Gate

from qiskit.quantum_info import Choi, Kraus
from qiskit.extensions import HGate, XGate
import qiskit.quantum_info.operators.channel.transformations as qtrans
import itertools
import csv
from qiskit.tools.monitor import job_monitor

from qiskit import QuantumCircuit, QuantumRegister,execute, Aer



#Definimos las matrices de Pauli
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])

In [2]:
# Primero iniciamos sesion
from qiskit import IBMQ
# Cargando cuenta de IBM (para realizar esto es necesaria una cuenta de IBM, preferiblemente esta sección se peude cargar desde IBM quantum lab)
provider = IBMQ.load_account()
provider = IBMQ.get_provider('ibm-q')

#Entrar aqui https://quantum-computing.ibm.com/
#con el correo @ciencias para ver los trabajos enviados y demás

C:\Users\tbasi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  after removing the cwd from sys.path.


In [3]:
qreg_q = QuantumRegister(3, 'q')  #3 registros cuánticos

circ = QuantumCircuit(qreg_q)
print(circ)

     
q_0: 
     
q_1: 
     
q_2: 
     


In [4]:
qpt_circs = process_tomography_circuits(circ,[0],prepared_qubits=[0])


TypeError: unsupported operand type(s) for +=: 'QuantumCircuit' and 'QuantumCircuit'